# Similarity search with Gemma Embeddings and Weaviate's `text2vec-transformers` module

This tutorial shows you, how to use the `gemma-3-300m-embedding` in Weaviate.

Model overview:

- Multilingual capabilities in over 100 languages
- Leverages Matryoshka Representation Learning for flexible output dimensions (128, 256, 512, 768)
- Maximum input sequence length of 8k tokens
- <300MB RAM usage with QAT

## Dependencies

In [1]:
%%capture
%pip install -q -U weaviate-client

## Connect to Weaviate

This Notebook uses the [`text2vec-transformers`](https://docs.weaviate.io/weaviate/model-providers/transformers/embeddings) module, which is **only** available through Weaviate open-source via [Docker](https://docs.weaviate.io/deploy/installation-guides/docker-installation) or [Kubernetes](https://docs.weaviate.io/deploy/installation-guides/k8s-installation). This integration is **not** available for Weaviate Cloud (WCD) serverless instances, as it requires spinning up a container with the Hugging Face model.

This notebook uses a [pre-built transformers model container](https://weaviate.io/developers/weaviate/modules/retriever-vectorizer-modules/text2vec-transformers#pre-built-images). For this, create a `docker-compose.yml` file with the following contents:
```
---
services:
  weaviate:
    command:
    - --host
    - 0.0.0.0
    - --port
    - '8080'
    - --scheme
    - http
    image: cr.weaviate.io/semitechnologies/weaviate:1.32.2
    ports:
    - 8080:8080
    - 50051:50051
    restart: on-failure:0
    environment:
      TRANSFORMERS_INFERENCE_API: 'http://t2v-transformers:8080'
      QUERY_DEFAULTS_LIMIT: 25
      AUTHENTICATION_ANONYMOUS_ACCESS_ENABLED: 'true'
      PERSISTENCE_DATA_PATH: '/var/lib/weaviate'
      DEFAULT_VECTORIZER_MODULE: 'text2vec-transformers'
      ENABLE_MODULES: 'text2vec-transformers'
      CLUSTER_HOSTNAME: 'weaviate-0'
  t2v-transformers:
    image: cr.weaviate.io/semitechnologies/transformers-inference:google-gemma-3-300m-embedding
    environment:
      ENABLE_CUDA: '0'
...
```
and start up Docker container with
```
docker-compose up -d
```

In [ ]:
import weaviate
from weaviate.config import AdditionalConfig

# Connect to your local Weaviate instance deployed with Docker
client = weaviate.connect_to_local(

)

client.is_ready()

/Users/leonie/Library/Python/3.9/lib/python/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.29.0 is exactly one major version older than the runtime version 6.31.1 at v1/aggregate.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
/Users/leonie/Library/Python/3.9/lib/python/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.29.0 is exactly one major version older than the runtime version 6.31.1 at v1/base.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
/Users/leonie/Library/Python/3.9/lib/python/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.29.0 is exactly one major version older than the runtime version 6.31.1 at v1/base_search.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warn

True

In [ ]:
# Check the cluster metadata to verify if the module is enabled
client.get_meta()['modules']

## Create a collection
> Collection stores your data and vector embeddings.

In [4]:
# Note: in practice, you shouldn't rerun this cell, as it deletes your data
# in "JeopardyQuestion", and then you need to re-import it again.
import weaviate.classes.config as wc

# Delete the collection if it already exists
if (client.collections.exists("JeopardyQuestion")):
    client.collections.delete("JeopardyQuestion")

client.collections.create(
    name="JeopardyQuestion",

    vector_config=wc.Configure.Vectors.text2vec_transformers( 
        dimensions=512, # default: 768, possible options: 128, 256, 512
    ),

    properties=[ # defining properties (data schema) is optional
        wc.Property(name="Question", data_type=wc.DataType.TEXT), 
        wc.Property(name="Answer", data_type=wc.DataType.TEXT),
        wc.Property(name="Category", data_type=wc.DataType.TEXT, skip_vectorization=True), 
    ]
)

print("Successfully created collection: JeopardyQuestion.")

Successfully created collection: JeopardyQuestion.


## Import the Data

In [5]:
import requests, json
url = 'https://raw.githubusercontent.com/weaviate/weaviate-examples/main/jeopardy_small_dataset/jeopardy_tiny.json'
resp = requests.get(url)
data = json.loads(resp.text)

# Get a collection object for "JeopardyQuestion"
jeopardy = client.collections.get("JeopardyQuestion")

# Insert data objects
with jeopardy.batch.fixed_size(
    batch_size=2,
    # concurrent_requests=1
) as batch:
    for item in data:
        print("Adding", item)
        batch.add_object(item)

print("Import Complete")

if (len(jeopardy.batch.failed_objects) > 0):
    print("There were some errors")
    for fo in jeopardy.batch.failed_objects:
        print(fo)

/Users/leonie/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Adding {'Category': 'SCIENCE', 'Question': 'This organ removes excess glucose from the blood & stores it as glycogen', 'Answer': 'Liver'}
Adding {'Category': 'ANIMALS', 'Question': "It's the only living mammal in the order Proboseidea", 'Answer': 'Elephant'}
Adding {'Category': 'ANIMALS', 'Question': 'The gavial looks very much like a crocodile except for this bodily feature', 'Answer': 'the nose or snout'}
Adding {'Category': 'ANIMALS', 'Question': 'Weighing around a ton, the eland is the largest species of this animal in Africa', 'Answer': 'Antelope'}
Adding {'Category': 'ANIMALS', 'Question': 'Heaviest of all poisonous snakes is this North American rattlesnake', 'Answer': 'the diamondback rattler'}
Adding {'Category': 'SCIENCE', 'Question': "2000 news: the Gunnison sage grouse isn't just another northern sage grouse, but a new one of this classification", 'Answer': 'species'}
Adding {'Category': 'SCIENCE', 'Question': 'A metal that is "ductile" can be pulled into this while cold & u

Quick check to see if all objects are in.

In [6]:
len(jeopardy)

10

## Query Weaviate: Similarity Search (Text objects)

Similarity search options for text objects in **Weaviate**:

1. [near_text](https://weaviate.io/developers/weaviate/search/similarity#an-input-medium)

2. [near_object](https://weaviate.io/developers/weaviate/search/similarity#an-object)

3. [near_vector](https://weaviate.io/developers/weaviate/search/similarity#a-vector)

### nearText Example

Find a `JeopardyQuestion` about "animals in movies". Limit it to only 4 responses.

In [ ]:
import weaviate.classes.query as wq

# note, you can reuse the collection object from the previous cell.
# Get a collection object for "JeopardyQuestion"
jeopardy = client.collections.get("JeopardyQuestion")

response = jeopardy.query.near_text(
    query="african beasts",
    include_vector=True,                             # return the vector embeddings
    return_metadata=wq.MetadataQuery(distance=True), # return the distance
    limit=4
)

for item in response.objects:
    print("ID:", item.uuid)
    print("Data:", json.dumps(item.properties, indent=2))
    print("Vector:", item.vector)
    print("Distance:", item.metadata.distance, "\n")

Let's check the length of one vector embedding:

In [8]:
len(item.vector['default'])

512

In [9]:
# Save last item's vector embedding and UUID for near_object and near_vector search
vector = item.vector['default']
uuid = item.uuid

### nearObject Example

Search through the `JeopardyQuestion` class to find the top 4 objects closest to UUID, we saved earlier.

In [10]:
print(f"Retrieve objects closest to item with UUID: {uuid}\n")

response = jeopardy.query.near_object(
    near_object=uuid, # replace with your id of interest
    limit=4
)

for item in response.objects:
    print("ID:", item.uuid)
    print("Data:", item.properties, "\n")

Retrieve objects closest to item with UUID: 1a63abbc-41c5-45da-b28e-819c58d14896

ID: 1a63abbc-41c5-45da-b28e-819c58d14896
Data: {'category': 'ANIMALS', 'question': 'Weighing around a ton, the eland is the largest species of this animal in Africa', 'answer': 'Antelope'} 

ID: 91bf08c7-131c-494d-9699-d374918bc88d
Data: {'question': "It's the only living mammal in the order Proboseidea", 'category': 'ANIMALS', 'answer': 'Elephant'} 

ID: 3c50fa78-6e61-4a32-b333-d99cba58ac0e
Data: {'question': 'Heaviest of all poisonous snakes is this North American rattlesnake', 'category': 'ANIMALS', 'answer': 'the diamondback rattler'} 

ID: 933505f9-176b-45da-891e-880b9a69c178
Data: {'category': 'SCIENCE', 'question': 'This organ removes excess glucose from the blood & stores it as glycogen', 'answer': 'Liver'} 



### nearVector Example

Search through the `JeopardyQuestion` class to find the top 2 objects closest to the query vector, we saved earlier.

In [11]:
print(f"Retrieve objects closest to item with vector: {vector[:5]}...\n")

response = jeopardy.query.near_vector(
    near_vector=vector, # your vector object goes here
    limit=4
)

for item in response.objects:
    print("ID:", item.uuid)
    print("Data:", item.properties, "\n")

Retrieve objects closest to item with vector: [0.0196092426776886, -0.02806108444929123, -0.017384057864546776, 0.02691761963069439, -0.03801804780960083]...

ID: 1a63abbc-41c5-45da-b28e-819c58d14896
Data: {'question': 'Weighing around a ton, the eland is the largest species of this animal in Africa', 'category': 'ANIMALS', 'answer': 'Antelope'} 

ID: 91bf08c7-131c-494d-9699-d374918bc88d
Data: {'question': "It's the only living mammal in the order Proboseidea", 'category': 'ANIMALS', 'answer': 'Elephant'} 

ID: 3c50fa78-6e61-4a32-b333-d99cba58ac0e
Data: {'question': 'Heaviest of all poisonous snakes is this North American rattlesnake', 'category': 'ANIMALS', 'answer': 'the diamondback rattler'} 

ID: 933505f9-176b-45da-891e-880b9a69c178
Data: {'question': 'This organ removes excess glucose from the blood & stores it as glycogen', 'category': 'SCIENCE', 'answer': 'Liver'} 

